In [48]:
import pandas as pd
import numpy as np

In [49]:
df = pd.read_csv("signals.csv", header=0, index_col=['timestamp'])

In [50]:
df.index = pd.to_datetime(df.index, unit='ms')
print(df.head())
df.dropna(axis=1, inplace=True)
df['mid_price'] = df['bid_price'].add(df['ask_price']) * 0.5

                         bid_price  ask_price  signal_0  signal_1  signal_2  \
timestamp                                                                     
2024-04-18 00:00:00.100    61256.0    61256.5 -0.000408 -0.000408 -0.001559   
2024-04-18 00:00:00.200    61256.0    61256.5 -0.000408 -0.000408 -0.001810   
2024-04-18 00:00:00.300    61256.0    61256.5 -0.000408 -0.001203 -0.002297   
2024-04-18 00:00:00.400    61256.0    61256.5 -0.000408 -0.001203 -0.002297   
2024-04-18 00:00:00.500    61256.0    61256.5 -0.000489 -0.002032 -0.003072   

                         signal_3  signal_4  signal_5  signal_6  signal_7  \
timestamp                                                                   
2024-04-18 00:00:00.100 -0.002500 -0.002501  0.005168  0.006319  0.008100   
2024-04-18 00:00:00.200 -0.002500 -0.002501  0.005168  0.006133  0.008015   
2024-04-18 00:00:00.300 -0.002759 -0.002760  0.001700  0.003524  0.005332   
2024-04-18 00:00:00.400 -0.002822 -0.002823  0.001377  0.0028

In [51]:
df = df.dropna(axis=0)

In [52]:
df = df.drop(['bid_price', 'ask_price'], axis=1)

In [53]:
df['mid_price'] = df['mid_price'].diff()
df['mid_price'] = df['mid_price'].shift(-1)

In [54]:
df = df.dropna(axis=0)
features = df.drop(['mid_price'], axis=1)

In [55]:
from sklearn.model_selection import train_test_split
df['target'] = np.sign(df['mid_price'])
features = df.drop(['mid_price', 'target'], axis=1)

In [56]:
columns = features.columns
for i in range(1, 6):
    for col in columns:
        features[col + "_" + str(i)] = features[col].shift(i)

/tmp/ipykernel_1957182/2275577595.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  features[col + "_" + str(i)] = features[col].shift(i)
/tmp/ipykernel_1957182/2275577595.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  features[col + "_" + str(i)] = features[col].shift(i)
/tmp/ipykernel_1957182/2275577595.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instea

In [57]:
from imblearn.over_sampling import SMOTE

features = features.dropna()
target = df['target'].shift(5).dropna()
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.4)
smote = SMOTE(random_state=42)
x_train, y_train = smote.fit_resample(x_train, y_train)

In [58]:
from sklearn.ensemble import RandomForestClassifier as RFC

In [59]:
rf = RFC(random_state=42, verbose=1, n_jobs=8)
rf.fit(x_train, y_train)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:   14.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:   40.3s finished


RandomForestClassifier(n_jobs=8, random_state=42, verbose=1)

In [60]:
from sklearn.metrics import classification_report, confusion_matrix

In [61]:
predictions = rf.predict(x_test)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.2s finished


In [62]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

        -1.0       0.66      0.69      0.68       415
         0.0       0.99      0.98      0.98     12141
         1.0       0.66      0.66      0.66       301

    accuracy                           0.97     12857
   macro avg       0.77      0.78      0.77     12857
weighted avg       0.97      0.97      0.97     12857



In [63]:
print((df['target'] == 0).sum())
print(df.shape)

30345
(32147, 32)
